In [11]:
import streamlit as st
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split

import time
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pickle

# Package for generating classification report
from sklearn.metrics import classification_report

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [12]:
input = pd.read_csv('fetal_health.csv')
X = input.drop(columns = 'fetal_health')
y = input['fetal_health']

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)
adaclassifier = AdaBoostClassifier( random_state = 1)
adaclassifier.fit(train_X,train_y)

ypred = adaclassifier.predict(test_X)

In [13]:
adacm = confusion_matrix(test_y, ypred, labels = adaclassifier.classes_)
adadisp = ConfusionMatrixDisplay(confusion_matrix = adacm, display_labels = adaclassifier.classes_)


fig, ax = plt.subplots(figsize = ((10, 8)))
plt.rcParams.update({'font.size': 12})


adadisp.plot(cmap = 'PuRd', ax = ax);
plt.savefig('adacm.svg')

In [17]:
adareport = classification_report(test_y, ypred, output_dict = True)
adareport_df = pd.DataFrame(adareport)
adareport_df.to_csv('ada_report.csv') 

In [15]:
importance = adaclassifier.feature_importances_

# Storing feature importance as a dataframe
feature_imp = pd.DataFrame(list(zip(train_X.columns, importance)),
               columns = ['Feature', 'Importance'])

feature_imp = feature_imp.sort_values('Importance', ascending = False).reset_index(drop = True)

# Bar plot
plt.figure(figsize=(10, 5))
plt.barh(feature_imp['Feature'], feature_imp['Importance'], color = ['purple', 'pink'])

plt.xlabel("Importance")
plt.ylabel("Input Feature")
plt.title('Which features affect price of mobiles') 
plt.tight_layout()
plt.savefig("adaimp.svg");

In [16]:
ada_pickle = open('adaboost.pickle', 'wb') 

# Write DT model to the file
pickle.dump(adaclassifier, ada_pickle) 

# Close the file
ada_pickle.close() 